# Train 3D Unet for vesicle segmentation

This notebook trains a 3D Unet deep learning network with vesicle segmentation data. The training set is created in the [create_training](create_training.ipynb) notebook.

## Set-up

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import tensorflow as tf

#from keras import backend as K
#K.set_image_data_format('channels_last')

# import unetmic.unetmic as umic
import prepyto.unetmic.unetmic.unetmic as umic

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
if tf.__version__[0] == '1':
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.keras.backend.set_session(tf.Session(config=config))
    print('Check if GPU is available')
    print(tf.test.gpu_device_name())
else:
    print('Check if GPU is available')
    print(tf.config.list_physical_devices())

Check if GPU is available
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# TF 2Check if a GPU is available
gpu_available = tf.config.list_physical_devices('GPU')
print("Is GPU available:", len(gpu_available) > 0)

Is GPU available: True


## Create network and train

In [4]:
#create the 3d-unet
unet3d = umic.create_unet_3d()

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
#specifiy path to training data and to folder where to save training weights
folder = '/mnt/data/amin/Data/train_dataset/'
save_folder = '/mnt/data/amin/Data/cryo_learning/'

#specify number of training data, validation data, batch size
numtot = 900
numvalid = 200
batchsize = 50

In [6]:
umic.run_training(network = unet3d, save_folder = save_folder, folder = folder, 
                numtot = numtot,batchsize = batchsize, numvalid = numvalid)

metric
Epoch 1/200
18/18 [==============================] - 6s 201ms/step - batch: 8.5000 - size: 50.0000 - loss: 0.5539 - dice_coef: 0.2292 - val_loss: 0.6145 - val_dice_coef: 0.2531
Epoch 2/200
18/18 [==============================] - 3s 172ms/step - batch: 8.5000 - size: 50.0000 - loss: 0.4246 - dice_coef: 0.2589 - val_loss: 0.6817 - val_dice_coef: 0.3224
Epoch 3/200
18/18 [==============================] - 3s 172ms/step - batch: 8.5000 - size: 50.0000 - loss: 0.3568 - dice_coef: 0.3378 - val_loss: 0.4772 - val_dice_coef: 0.3563
Epoch 4/200
18/18 [==============================] - 3s 171ms/step - batch: 8.5000 - size: 50.0000 - loss: 0.3018 - dice_coef: 0.4205 - val_loss: 0.3584 - val_dice_coef: 0.4036
Epoch 5/200
18/18 [==============================] - 3s 172ms/step - batch: 8.5000 - size: 50.0000 - loss: 0.2678 - dice_coef: 0.4668 - val_loss: 0.3493 - val_dice_coef: 0.4564
Epoch 6/200
18/18 [==============================] - 3s 171ms/step - batch: 8.5000 - size: 50.0000 - loss: 0